In [2]:
import pyspark

In [3]:
pyspark.__version__

'3.0.3'

In [7]:
from pyspark.sql import SparkSession

In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-26 22:00:30--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.42.52
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.42.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  6.19MB/s    in 2m 8s   

2022-02-26 22:02:39 (5.59 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [5]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [6]:
!head fhvhv_tripdata_2021-01.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,


In [8]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/26 22:04:31 WARN Utils: Your hostname, HGDEWLFR001442 resolves to a loopback address: 127.0.1.1; using 172.17.181.93 instead (on interface eth0)
22/02/26 22:04:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/26 22:04:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/26 22:04:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [12]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [13]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [16]:
!wc -l head.csv

101 head.csv


In [17]:
import pandas as pd

In [18]:
df_pandas = pd.read_csv('head.csv')

In [22]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [26]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [29]:
from pyspark.sql import types

In [40]:
schema = types.StructType([
    types.StructField('hvfhs_license_num',types.StringType(),True),
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.TimestampType(),True),
    types.StructField('dropoff_datetime',types.TimestampType(),True),
    types.StructField('PULocationID',types.IntegerType(),True),
    types.StructField('DOLocationID',types.IntegerType(),True),
    types.StructField('SR_Flag',types.StringType(),True)
])

In [33]:
schema2 = 'string,string,string,string,int,int,string'

In [41]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [42]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [45]:
df = df.repartition(24)

In [47]:
df.write.parquet('dhvhv/2021/01/')

AnalysisException: path file:/home/konjufcad/code/de-course/work/week4_batch_processing/dhvhv/2021/01 already exists.;

In [48]:
df = spark.read.parquet('dhvhv/2021/01/')

In [51]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [56]:
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID') \
.filter(df.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 10:18:58|2021-01-03 10:44:37|         142|         138|
|2021-01-01 17:07:47|2021-01-01 17:24:05|         213|         265|
|2021-01-01 17:40:20|2021-01-01 17:54:23|         255|          97|
|2021-01-01 16:05:08|2021-01-01 16:08:50|          62|          61|
|2021-01-01 10:45:44|2021-01-01 10:53:22|          14|         228|
|2021-01-01 10:32:57|2021-01-01 10:52:13|         181|         125|
|2021-01-02 20:01:47|2021-01-02 20:31:26|          33|          35|
|2021-01-03 01:53:03|2021-01-03 02:00:44|          60|         182|
|2021-01-01 05:54:34|2021-01-01 06:18:28|          65|          39|
|2021-01-01 17:35:19|2021-01-01 17:45:22|         141|         236|
|2021-01-01 01:08:23|2021-01-01 01:25:36|          91|          76|
|2021-01-01 16:52:13|2021-01-01 17:08:46|       

In [57]:
from pyspark.sql import functions as F

In [66]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [67]:
crazy_stuff('B02884')

's/b44'

In [80]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [82]:
df\
.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
.withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
.withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
.select('base_id','pickup_date','dropoff_date','PULocationID','DOLocationID') \
.show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  s/b44| 2021-01-03|  2021-01-03|         142|         138|
|  e/acc| 2021-01-01|  2021-01-01|         213|         265|
|  e/9ce| 2021-01-01|  2021-01-01|         156|         147|
|  e/acc| 2021-01-01|  2021-01-01|         255|          97|
|  e/b38| 2021-01-01|  2021-01-01|          62|          61|
|  a/b37| 2021-01-01|  2021-01-01|          14|         228|
|  e/b3b| 2021-01-01|  2021-01-01|         181|         125|
|  e/b48| 2021-01-02|  2021-01-02|          33|          35|
|  e/b3f| 2021-01-03|  2021-01-03|          60|         182|
|  e/acc| 2021-01-01|  2021-01-01|          65|          39|
|  e/b3b| 2021-01-01|  2021-01-01|         141|         236|
|  e/b38| 2021-01-01|  2021-01-01|          91|          76|
|  a/b37| 2021-01-01|  2021-01-01|         134|         191|
|  a/a7a| 2021-01-01|  2

Traceback (most recent call last):
  File "/home/konjufcad/spark/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/home/konjufcad/spark/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/home/konjufcad/spark/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 642, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/home/konjufcad/spark/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError
